In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

In [4]:
df = pd.read_csv('../dataset/sonar.csv', header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M


In [5]:
dataset = df.values
X = dataset[:,0:60]
Y_obj = dataset[:,60]
print(X)
print(Y_obj)

[[0.02 0.0371 0.0428 ... 0.0084 0.009 0.0032]
 [0.0453 0.0523 0.0843 ... 0.0049 0.0052 0.0044]
 [0.0262 0.0582 0.1099 ... 0.0164 0.0095 0.0078]
 ...
 [0.0522 0.0437 0.018 ... 0.0138 0.0077 0.0031]
 [0.0303 0.0353 0.049 ... 0.0079 0.0036 0.0048]
 [0.026 0.0363 0.0136 ... 0.0036 0.0061 0.0115]]
['R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 

In [6]:
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)
print(Y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [8]:
n_fold = 10
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [9]:
X = X.astype(np.float32)

In [10]:
accuracy = []

In [17]:
for train, test in skf.split(X, Y):
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error',
                 optimizer='adam',
                 metrics=['accuracy'])
    model.fit(X[train], Y[train], epochs=100, batch_size=5)
    k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)

Train on 187 samples
Epoch 1/100
187/187 [==============================] - 0s 2ms/sample - loss: 0.2582 - accuracy: 0.4545
Epoch 2/100
187/187 [==============================] - 0s 261us/sample - loss: 0.2403 - accuracy: 0.6150
Epoch 3/100
187/187 [==============================] - 0s 257us/sample - loss: 0.2339 - accuracy: 0.6578
Epoch 4/100
187/187 [==============================] - 0s 267us/sample - loss: 0.2263 - accuracy: 0.6791
Epoch 5/100
187/187 [==============================] - 0s 251us/sample - loss: 0.2173 - accuracy: 0.7219
Epoch 6/100
187/187 [==============================] - 0s 245us/sample - loss: 0.2078 - accuracy: 0.7380
Epoch 7/100
187/187 [==============================] - 0s 245us/sample - loss: 0.1980 - accuracy: 0.7380
Epoch 8/100
187/187 [==============================] - 0s 267us/sample - loss: 0.1938 - accuracy: 0.7273
Epoch 9/100
187/187 [==============================] - 0s 283us/sample - loss: 0.1799 - accuracy: 0.7647
Epoch 10/100
187/187 [==============

187/187 [==============================] - 0s 224us/sample - loss: 0.0417 - accuracy: 0.9572
Epoch 79/100
187/187 [==============================] - 0s 251us/sample - loss: 0.0393 - accuracy: 0.9679
Epoch 80/100
187/187 [==============================] - 0s 256us/sample - loss: 0.0440 - accuracy: 0.9465
Epoch 81/100
187/187 [==============================] - 0s 261us/sample - loss: 0.0400 - accuracy: 0.9572
Epoch 82/100
187/187 [==============================] - 0s 235us/sample - loss: 0.0379 - accuracy: 0.9679
Epoch 83/100
187/187 [==============================] - 0s 213us/sample - loss: 0.0353 - accuracy: 0.9626
Epoch 84/100
187/187 [==============================] - 0s 213us/sample - loss: 0.0364 - accuracy: 0.9679
Epoch 85/100
187/187 [==============================] - 0s 219us/sample - loss: 0.0350 - accuracy: 0.9679
Epoch 86/100
187/187 [==============================] - 0s 224us/sample - loss: 0.0334 - accuracy: 0.9626
Epoch 87/100
187/187 [==============================] - 0s 

187/187 [==============================] - 0s 251us/sample - loss: 0.0697 - accuracy: 0.9198
Epoch 55/100
187/187 [==============================] - 0s 272us/sample - loss: 0.0638 - accuracy: 0.9465
Epoch 56/100
187/187 [==============================] - 0s 288us/sample - loss: 0.0620 - accuracy: 0.9465
Epoch 57/100
187/187 [==============================] - 0s 320us/sample - loss: 0.0617 - accuracy: 0.9465
Epoch 58/100
187/187 [==============================] - 0s 283us/sample - loss: 0.0623 - accuracy: 0.9412
Epoch 59/100
187/187 [==============================] - 0s 261us/sample - loss: 0.0585 - accuracy: 0.9412
Epoch 60/100
187/187 [==============================] - 0s 261us/sample - loss: 0.0555 - accuracy: 0.9679
Epoch 61/100
187/187 [==============================] - 0s 256us/sample - loss: 0.0546 - accuracy: 0.9572
Epoch 62/100
187/187 [==============================] - 0s 261us/sample - loss: 0.0548 - accuracy: 0.9519
Epoch 63/100
187/187 [==============================] - 0s 

187/187 [==============================] - 0s 251us/sample - loss: 0.1004 - accuracy: 0.8930
Epoch 31/100
187/187 [==============================] - 0s 256us/sample - loss: 0.0998 - accuracy: 0.8770
Epoch 32/100
187/187 [==============================] - 0s 251us/sample - loss: 0.0964 - accuracy: 0.8984
Epoch 33/100
187/187 [==============================] - 0s 288us/sample - loss: 0.0973 - accuracy: 0.8984
Epoch 34/100
187/187 [==============================] - 0s 384us/sample - loss: 0.0917 - accuracy: 0.9091
Epoch 35/100
187/187 [==============================] - 0s 288us/sample - loss: 0.0935 - accuracy: 0.8930
Epoch 36/100
187/187 [==============================] - 0s 251us/sample - loss: 0.0883 - accuracy: 0.9198
Epoch 37/100
187/187 [==============================] - 0s 256us/sample - loss: 0.0862 - accuracy: 0.9037
Epoch 38/100
187/187 [==============================] - 0s 256us/sample - loss: 0.0915 - accuracy: 0.8824
Epoch 39/100
187/187 [==============================] - 0s 

187/187 [==============================] - 0s 288us/sample - loss: 0.2091 - accuracy: 0.7326
Epoch 7/100
187/187 [==============================] - 0s 283us/sample - loss: 0.2000 - accuracy: 0.7594
Epoch 8/100
187/187 [==============================] - 0s 283us/sample - loss: 0.1990 - accuracy: 0.7166
Epoch 9/100
187/187 [==============================] - 0s 267us/sample - loss: 0.1867 - accuracy: 0.7487
Epoch 10/100
187/187 [==============================] - 0s 251us/sample - loss: 0.1740 - accuracy: 0.8075
Epoch 11/100
187/187 [==============================] - 0s 245us/sample - loss: 0.1693 - accuracy: 0.7647
Epoch 12/100
187/187 [==============================] - 0s 251us/sample - loss: 0.1690 - accuracy: 0.7861
Epoch 13/100
187/187 [==============================] - 0s 256us/sample - loss: 0.1620 - accuracy: 0.8075
Epoch 14/100
187/187 [==============================] - 0s 256us/sample - loss: 0.1519 - accuracy: 0.7914
Epoch 15/100
187/187 [==============================] - 0s 261

...

187/187 [==============================] - 0s 240us/sample - loss: 0.0372 - accuracy: 0.9733
Epoch 89/100
187/187 [==============================] - 0s 240us/sample - loss: 0.0377 - accuracy: 0.9679
Epoch 90/100
187/187 [==============================] - 0s 224us/sample - loss: 0.0375 - accuracy: 0.9626
Epoch 91/100
187/187 [==============================] - 0s 224us/sample - loss: 0.0356 - accuracy: 0.9733
Epoch 92/100
187/187 [==============================] - 0s 224us/sample - loss: 0.0360 - accuracy: 0.9733
Epoch 93/100
187/187 [==============================] - 0s 224us/sample - loss: 0.0344 - accuracy: 0.9679
Epoch 94/100
187/187 [==============================] - 0s 235us/sample - loss: 0.0330 - accuracy: 0.9840
Epoch 95/100
187/187 [==============================] - 0s 229us/sample - loss: 0.0323 - accuracy: 0.9733
Epoch 96/100
187/187 [==============================] - 0s 224us/sample - loss: 0.0327 - accuracy: 0.9679
Epoch 97/100
187/187 [==============================] - 0s 

187/187 [==============================] - 0s 251us/sample - loss: 0.0492 - accuracy: 0.9465
Epoch 65/100
187/187 [==============================] - 0s 245us/sample - loss: 0.0476 - accuracy: 0.9465
Epoch 66/100
187/187 [==============================] - 0s 240us/sample - loss: 0.0495 - accuracy: 0.9626
Epoch 67/100
187/187 [==============================] - 0s 235us/sample - loss: 0.0467 - accuracy: 0.9519
Epoch 68/100
187/187 [==============================] - 0s 240us/sample - loss: 0.0456 - accuracy: 0.9412
Epoch 69/100
187/187 [==============================] - 0s 229us/sample - loss: 0.0420 - accuracy: 0.9572
Epoch 70/100
187/187 [==============================] - 0s 240us/sample - loss: 0.0420 - accuracy: 0.9572
Epoch 71/100
187/187 [==============================] - 0s 240us/sample - loss: 0.0415 - accuracy: 0.9626
Epoch 72/100
187/187 [==============================] - 0s 240us/sample - loss: 0.0384 - accuracy: 0.9786
Epoch 73/100
187/187 [==============================] - 0s 

188/188 [==============================] - 0s 228us/sample - loss: 0.0762 - accuracy: 0.9096
Epoch 41/100
188/188 [==============================] - 0s 223us/sample - loss: 0.0718 - accuracy: 0.9202
Epoch 42/100
188/188 [==============================] - 0s 239us/sample - loss: 0.0685 - accuracy: 0.9415
Epoch 43/100
188/188 [==============================] - 0s 228us/sample - loss: 0.0698 - accuracy: 0.9202
Epoch 44/100
188/188 [==============================] - 0s 223us/sample - loss: 0.0660 - accuracy: 0.9415
Epoch 45/100
188/188 [==============================] - 0s 228us/sample - loss: 0.0663 - accuracy: 0.9362
Epoch 46/100
188/188 [==============================] - 0s 233us/sample - loss: 0.0618 - accuracy: 0.9309
Epoch 47/100
188/188 [==============================] - 0s 228us/sample - loss: 0.0594 - accuracy: 0.9574
Epoch 48/100
188/188 [==============================] - 0s 228us/sample - loss: 0.0599 - accuracy: 0.9415
Epoch 49/100
188/188 [==============================] - 0s 

188/188 [==============================] - 0s 260us/sample - loss: 0.1502 - accuracy: 0.7872
Epoch 17/100
188/188 [==============================] - 0s 249us/sample - loss: 0.1483 - accuracy: 0.7872
Epoch 18/100
188/188 [==============================] - 0s 239us/sample - loss: 0.1393 - accuracy: 0.8191
Epoch 19/100
188/188 [==============================] - 0s 260us/sample - loss: 0.1395 - accuracy: 0.8191
Epoch 20/100
188/188 [==============================] - 0s 271us/sample - loss: 0.1362 - accuracy: 0.8245
Epoch 21/100
188/188 [==============================] - 0s 239us/sample - loss: 0.1303 - accuracy: 0.8245
Epoch 22/100
188/188 [==============================] - 0s 281us/sample - loss: 0.1257 - accuracy: 0.8723
Epoch 23/100
188/188 [==============================] - 0s 228us/sample - loss: 0.1265 - accuracy: 0.8404
Epoch 24/100
188/188 [==============================] - 0s 265us/sample - loss: 0.1214 - accuracy: 0.8511
Epoch 25/100
188/188 [==============================] - 0s 

188/188 [==============================] - 0s 228us/sample - loss: 0.0308 - accuracy: 0.9787
Epoch 94/100
188/188 [==============================] - 0s 228us/sample - loss: 0.0331 - accuracy: 0.9734
Epoch 95/100
188/188 [==============================] - 0s 228us/sample - loss: 0.0331 - accuracy: 0.9734
Epoch 96/100
188/188 [==============================] - 0s 228us/sample - loss: 0.0309 - accuracy: 0.9840
Epoch 97/100
188/188 [==============================] - 0s 233us/sample - loss: 0.0299 - accuracy: 0.9840
Epoch 98/100
188/188 [==============================] - 0s 228us/sample - loss: 0.0315 - accuracy: 0.9840
Epoch 99/100
188/188 [==============================] - 0s 223us/sample - loss: 0.0313 - accuracy: 0.9787
Epoch 100/100
20/20 [==============================] - 0s 4ms/sample - loss: 0.1234 - accuracy: 0.8500


In [18]:
print("\n %.f fold accuracy:"% n_fold, accuracy)


 10 fold accuracy: ['0.7619', '0.9048', '0.7619', '0.8095', '0.8095', '0.7619', '0.8571', '0.8571', '0.9500', '0.8500']


In [30]:
result = 0
for i in accuracy:
    result += float(i)
    if i == accuracy[-1]:
        result /= len(accuracy)
result

0.83237